In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv('.env')
directory = os.getenv("DIRECTORY")
sys.path.append(directory)
import pandas as pd
from qiita_db.study import Study, StudyPerson

# Get all public study IDs
study_ids = Study.get_ids_by_status('sandbox')
print(f"Found {len(study_ids)} studies")

# Get info for all studies 
study_info = Study.get_info(study_ids=list(study_ids))

# Convert to DataFrame
df = pd.DataFrame(study_info)

# Search for studies containing "soil" in title or abstract
search_term = 'harry'
results = df[
    df['study_title'].str.contains(search_term, case=False, na=False) |
    df['study_abstract'].fillna('').str.contains(search_term, case=False, na=False)
]

# Display results
print(f"\nFound {len(results)} studies matching '{search_term}'")
for _, row in results.iterrows():
    print(f"\nStudy {row['study_id']}: {row['study_title']}")
    if pd.notna(row['principal_investigator_id']):
        pi = StudyPerson(row['principal_investigator_id'])
        print(f"  PI: {pi.name}")
        print(f"  Abstract: {row['study_abstract'][:200]}...")

/Users/dhruvsharma/Documents/GitHub/qiita/qiita_core/configuration_manager.py:267: UserWarning: Using the github fake email for HELP_EMAIL, are you sure this is OK?
  warnings.warn(
/Users/dhruvsharma/Documents/GitHub/qiita/qiita_core/configuration_manager.py:280: UserWarning: Using the github fake email for SYSADMIN_EMAIL, are you sure this is OK?
  warnings.warn(


Found 4 studies

Found 4 studies matching 'harry'

Study 2: Harry Potter and the Philospher's Stone
  PI: LabDude
  Abstract: At Hogwarts, Harry learns about his parents’ mysterious deaths and his connection to the dark wizard Lord Voldemort. With his new friends Ron and Hermione, he uncovers the secret of the Sorcerer’s Sto...

Study 5: Harry Potter and the Goblet of Fire
  PI: PIDude
  Abstract: The Triwizard Tournament brings foreign wizarding schools to Hogwarts. Despite being underage, Harry is entered under mysterious circumstances. The competition ends tragically, and Voldemort returns t...

Study 4: Harry Potter and the Prisoner of Azkaban
  PI: PIDude
  Abstract: Sirius Black escapes from Azkaban, believed to be a dangerous follower of Voldemort. As Harry learns about Dementors, time travel, and his parents’ friends, the truth emerges: Sirius is innocent and d...

Study 3: Harry Potter and the Chamber of Secrets
  PI: empDude
  Abstract: During Harry’s second year, students ar

In [2]:
# query_studies.py - Run this in your Jupyter notebook

from qiita_db.study import Study
import pandas as pd

def search_studies(search_term=None, status='sandbox'):
    """
    Search Qiita studies by title, PI, or abstract
    
    Parameters
    ----------
    search_term : str, optional
        Term to search for across title, PI, and abstract
    status : str
        Study status ('public', 'private', 'sandbox', etc.)
    
    Returns
    -------
    pd.DataFrame
        DataFrame with study information
    """
    # Get study IDs by status
    study_ids = list(Study.get_ids_by_status(status))
    
    if not study_ids:
        print(f"No {status} studies found")
        return pd.DataFrame()
    
    # Get detailed info
    study_info = Study.get_info(study_ids)
    
    # Convert to DataFrame
    df = pd.DataFrame(study_info)
    
    if search_term and not df.empty:
        # Search across multiple fields
        mask = (
            df['study_title'].str.contains(search_term, case=False, na=False) |
            df['study_abstract'].astype(str).str.contains(search_term, case=False, na=False) |
            df.get('principal_investigator', pd.Series()).astype(str).str.contains(search_term, case=False, na=False)
        )
        df = df[mask]
    
    return df

# Example usage:
# Search for all studies containing "soil"
results = search_studies(search_term='voldemort')
print(f"Found {len(results)} studies")
print(results[['study_id', 'study_title']].head())
# print(results)
# Get specific study details
if not results.empty:
    study_id = results.iloc[0]['study_id']
    study = Study(int(study_id))
    print(f"\nStudy {study_id}: {study.title}")
    print(f"PI: {study.info['principal_investigator'].name if study.info['principal_investigator'] else 'N/A'}")
    print(f"Abstract: {study.info.get('study_abstract', 'N/A')[:200]}...")

Found 4 studies
   study_id                               study_title
0         2   Harry Potter and the Philospher's Stone
1         5       Harry Potter and the Goblet of Fire
2         4  Harry Potter and the Prisoner of Azkaban
3         3   Harry Potter and the Chamber of Secrets

Study 2: Harry Potter and the Philospher's Stone
PI: LabDude
Abstract: At Hogwarts, Harry learns about his parents’ mysterious deaths and his connection to the dark wizard Lord Voldemort. With his new friends Ron and Hermione, he uncovers the secret of the Sorcerer’s Sto...


/var/folders/sr/gw7824c55pq1_7gtbpkx37kr0000gn/T/ipykernel_46547/3776795205.py:40: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  df.get('principal_investigator', pd.Series()).astype(str).str.contains(search_term, case=False, na=False)
